# Camera Calibration

Code to take images of a checkerboard and obtain the instrinsic parameters of the camera

DISCALIMER: code inspired from the explainetions and code found at : https://learnopencv.com/camera-calibration-using-opencv/

imports: 

In [6]:
#!/usr/bin/env python
 
import cv2
import numpy as np
import os
import glob

Define checkerboard size and point vectors

In [7]:
CHECKERBOARD = (6,8)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

Take images or select them from memory if any

In [24]:
curr_path = os.getcwd()
print(curr_path)
images_path = os.path.join(curr_path, 'calibration_images')

if os.path.exists(os.path.join(curr_path,  'calibration_images')):
    # Extracting path of individual image stored in a given directory
    print("calibration images folder found")
    
else : 
    os.mkdir(images_path)

    # calibration images don't exist so lets take pictures
    print("calibration images folder not found. Let's take a set of 10 pictures of the checkboard")
    # Create the object to use the webcam
    gst_str = "nvarguscamerasrc ! video/x-raw(memory:NVMM)," \
                "width=(int)1280, height=(int)720, format=(string)NV12, " \
                "framerate=1/1! nvvidconv ! video/x-raw, " \
                "width=(int)1280, height=(int)720, format=(string)BGRx ! " \
                "videoconvert ! video/x-raw, format=(string)BGR !" \
                "appsink"
    vid = cv2.VideoCapture(gst_str, cv2.CAP_GSTREAMER)
    cv2.namedWindow("press space to capture image")
    img_counter = 0
    # Read a frame from your webcam
    while True:
        
        result, frame = vid.read()
        if not result:
            print("failed to grab frame")
            break
        cv2.imshow('press space to capture image', frame)
        k = cv2.waitKey(1)
        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            # SPACE pressed
            
            cv2.imwrite(os.path.join(images_path, "calibration_image_" + str(img_counter) +".jpg"), frame)
            print(" image written!")
            img_counter += 1
            if img_counter > 9:
                break

images = glob.glob(os.path.join(images_path , '*.jpg'))
# Close the webcam
vid.release()            
cv2.destroyAllWindows()

/home/nvidia/Desktop/projects
calibration images folder found
['/home/nvidia/Desktop/projects/calibration_images/calibration_image_0.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_1.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_2.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_5.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_6.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_8.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_9.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_7.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_3.jpg', '/home/nvidia/Desktop/projects/calibration_images/calibration_image_4.jpg']
here


extract points from images

In [25]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    
    cv2.imshow('img',img)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

Calibration function and show camera properties

In [26]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[ 949.08854289    0.          666.31415728]
 [   0.          951.10897066  369.93327023]
 [   0.            0.            1.        ]]
dist : 

[[  1.31223609e-01  -2.82456203e-01   4.00777386e-03   9.31164684e-05
    6.70004585e-02]]
rvecs : 

[array([[-0.14640488],
       [-0.25136172],
       [-1.50007839]]), array([[-0.13557091],
       [-0.3831708 ],
       [-1.52603872]]), array([[-0.69085935],
       [ 0.23368046],
       [-1.26155512]]), array([[ 0.08987701],
       [-0.27099053],
       [-1.45743478]]), array([[-0.13699981],
       [-0.20853942],
       [-1.37314104]]), array([[-0.17442714],
       [-0.00292975],
       [-1.52411247]]), array([[-0.35166302],
       [ 0.2336462 ],
       [-1.32407352]]), array([[-0.03102687],
       [ 0.35290343],
       [ 1.49154998]]), array([[-0.2601516 ],
       [-0.16355316],
       [-1.43301135]]), array([[ 0.40956747],
       [-0.77110543],
       [-1.49628555]])]
tvecs : 

[array([[ -4.28493489],
       [  1.04238129]